# Spark Datasources


## Postgres setup


1. Make SQL files accessible by postgres user

```bash
sudo cp /home/ubuntu/obenkyo/raw_data/spark_course_udemy/demo/* /tmp/
sudo chown postgres:postgres /tmp/1.CreateTable.sql
# from shell:
sudo -u postgres psql -d vendas -f /tmp/1.CreateTable.sql
# or inside psql:
\i /tmp/1.CreateTable.sql
```


2. Login and create database

```
sudo -u postgres sql
```

```
create database vendas;
```

```
\c vendas
```


3. Run queries

```psql
\i /tmp/1.CreateTable.sql
\i /tmp/2.InsertClientes.sql
\i /tmp/3.InsertIntoProdutos.sql
\i /tmp/4.InsertIntoVendedores.sql
\i /tmp/5.InsertIntoVendas.sql
\i /tmp/6.InsertItensVenda.sql
```

## JDBC

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PostgreSQL Example") \
    .config("spark.jars", "/home/ubuntu/obenkyo/raw_data/postgresql-42.7.8.jar") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/04 08:39:04 WARN Utils: Your hostname, WIN-NJTBBD8GS0T, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/04 08:39:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/11/04 08:39:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
vendas = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vendas") \
    .option("dbtable", "Vendas") \
    .option("user", "postgres") \
    .option("password", "123") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [13]:
vendas \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/vendas") \
    .option("dbtable", "Vendadata") \
    .option("user", "postgres") \
    .option("password", "123") \
    .option("driver", "org.postgresql.Driver") \
    .save()

# MongoDB

* Install

    * Follow: https://www.mongodb.com/docs/v7.0/tutorial/install-mongodb-on-ubuntu/


* First steps


```bash
sudo systemctl start mongod

mongoimport --db posts --collection post --legacy --file /home/ubuntu/obenkyo/raw_data/spark_course_udemy/mongo/posts.json

mongosh

show dbs

use posts  # database name

db.post.find()
```

In [2]:
spark

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("spark-mongo-local")
    # sessão genérica: sem DB/collection fixos
    .config("spark.mongodb.read.connection.uri",  "mongodb://127.0.0.1:27017")
    .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1:27017")
    .getOrCreate()
)

# escolha o DB/collection apenas na leitura
posts = (spark.read
         .format("mongodb")
         .option("database", "posts")
         .option("collection", "post")
         .load())
posts.show(5, truncate=False)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/05 08:32:55 WARN Utils: Your hostname, WIN-NJTBBD8GS0T, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/05 08:32:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/05 08:32:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/05 08:33:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Py4JJavaError: An error occurred while calling o35.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: mongodb. Make sure the provider name is correct and the package is properly registered and compatible with your Spark version. SQLSTATE: 42K02
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:681)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:740)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$apply$1.applyOrElse(ResolveDataSource.scala:58)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$apply$1.applyOrElse(ResolveDataSource.scala:45)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$3(AnalysisHelper.scala:139)
	at org.apache.spark.s